In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install --upgrade accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 17.9 MB/s eta 0:00:00


In [ ]:
!pip install transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 121.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 123.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 54.1 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.12.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 3.20.2 which is incompatible.
ten

In [ ]:
!pip install datasets transformers seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 65.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 11.7 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16165 sha256=fe7ca5b8172b911a2a22c52b6302ac722eaf85bd6c40238

In [ ]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 9.7 MB/s eta 0:00:00


In [ ]:
#get dataset
from datasets import load_dataset
mnli_data = load_dataset("multi_nli")

Generating train split:   0%|          | 0/392702 [00:00<?, ? examples/s]

Generating validation_matched split:   0%|          | 0/9815 [00:00<?, ? examples/s]

Generating validation_mismatched split:   0%|          | 0/9832 [00:00<?, ? examples/s]

Dataset multi_nli downloaded and prepared to /root/.cache/huggingface/datasets/multi_nli/default/0.0.0/591f72eb6263d1ab527561777936b199b714cda156d35716881158a2bd144f39. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
#import tokenizer
from transformers import  BertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-cased") 

In [ ]:
#preprocessing
def preprocess_function(examples):
    return tokenizer(examples['premise'], examples['hypothesis'], truncation=True)
#set format
mnli_data = mnli_data.map(preprocess_function, batched=True)
mnli_data.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

Map:   0%|          | 0/392702 [00:00<?, ? examples/s]

Map:   0%|          | 0/9815 [00:00<?, ? examples/s]

Map:   0%|          | 0/9832 [00:00<?, ? examples/s]

In [ ]:
#select a sample and tokenize
sample_size = 200000
train_dataset = mnli_data['train'].shuffle().select(range(sample_size))
eval_dataset=  mnli_data['validation_matched'].shuffle().select(range(5000))

In [ ]:
import transformers

In [ ]:
#get data collator
data_collator = transformers.DataCollatorWithPadding(tokenizer=tokenizer, padding=True)

In [ ]:
#get model
checkpoint = "bert-base-cased"
#define model
from transformers import  BertForSequenceClassification, TrainingArguments, Trainer
num_labels  = 3 #for nli task
model = BertForSequenceClassification.from_pretrained(checkpoint, 
                                                        num_labels=num_labels, 
                                                        ignore_mismatched_sizes=True)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [ ]:
from transformers import TrainingArguments, Trainer

checkpoint_path = '/content/drive/MyDrive/results/bert/checkpoint-25000'

model = BertForSequenceClassification.from_pretrained(checkpoint_path)

In [ ]:
#define training args
training_args = TrainingArguments(
    output_dir='drive/MyDrive/results/bert',  # Output directory
    evaluation_strategy='epoch',  # Evaluation strategy to adopt during training
    learning_rate=2e-5,  # Learning rate
    per_device_train_batch_size=16,  # Batch size per device during training
    per_device_eval_batch_size=32,   # Batch size for evaluation
    num_train_epochs=2,  # Number of training epochs
    weight_decay=0.01,  # Weight decay
    push_to_hub=False,
)

In [ ]:
trainer = Trainer(
    model,  # The instantiated 🤗 Transformers model to be trained
    args=training_args,  # Training arguments, defined above
    train_dataset= train_dataset,  # Training dataset
    eval_dataset= eval_dataset,  # Evaluation dataset
    data_collator= data_collator,  # Optional function to apply to the input data before feeding it to the model
    # compute_metrics=compute_metrics,
)
evaluation = trainer.evaluate()



In [ ]:
import numpy as np

predictions = trainer.predict(eval_dataset)
print(predictions.predictions.shape, predictions.label_ids.shape)
preds = np.argmax(predictions.predictions, axis=-1)

(5000, 3) (5000,)


In [ ]:
import evaluate

metric = evaluate.load("f1")
print(metric.compute(predictions=preds, references=predictions.label_ids, average="weighted"))


metric = evaluate.load("glue", "mnli")
print(metric.compute(predictions=preds, references=predictions.label_ids))

{'f1': 0.8245622644640817}


{'accuracy': 0.8242}


In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.427200,0.388541
2,0.297500,0.405574


TrainOutput(global_step=25000, training_loss=0.3946479010009766, metrics={'train_runtime': 7332.6534, 'train_samples_per_second': 54.551, 'train_steps_per_second': 3.409, 'total_flos': 1.6616911452023232e+16, 'train_loss': 0.3946479010009766, 'epoch': 2.0})